In [1]:
from arcgis.gis import GIS
import json
import requests
from requests.auth import HTTPBasicAuth
from getpass import getpass
from IPython.display import display


## Debug Flag
debug = True
## Temporary File Name
pega_temp = "pega_response.geojson"

In [2]:
#### Pega URL Request
pega_REST = 'http://irsecm.pegatsdemo.com:8080/prweb/PRRestService/esriapi/v1/casesGeojson'
pega_user = 'esri.restAPI'
pega_password = getpass()

pega_response = requests.get(pega_REST, auth=HTTPBasicAuth(pega_user, pega_password))
pega_file = open(pega_temp, "w")
pega_file.write(str(pega_response.json()))
pega_file.close()

if(debug):
    print(pega_response.json())

········
{'features': [{'properties': {'caseRef': 'IRS-SAMPLEAP-WORK M-88', 'createdBy': 'Alexei Ivanov', 'lastUpdateTime': '2017-07-17T13:26:01.005Z', 'name': 'Missing Refund', 'caseType': 'IRS_SampleAp_Work_MissingRefund', 'createTime': '2017-07-17T13:24:11.512Z', 'lastUpdatedBy': 'Alexei Ivanov'}, 'geometry': {'coordinates': [-77.1823399, 39.1498103], 'type': 'Point'}, 'type': 'Feature'}, {'properties': {'caseRef': 'IRS-SAMPLEAP-WORK R-31', 'createdBy': 'Alexei Ivanov', 'lastUpdateTime': '2017-07-31T13:26:26.570Z', 'name': 'Return Investigation', 'caseType': 'IRS_SampleAp_Work_ReturnInvestigation', 'createTime': '2017-07-17T13:26:15.847Z', 'lastUpdatedBy': 'Alexei Ivanov'}, 'geometry': {'coordinates': [-77.18234319999999, 39.1498083], 'type': 'Point'}, 'type': 'Feature'}], 'type': 'FeatureCollection'}


In [ ]:
#### Connect to ArcGIS Online
agol_url = 'https://seankmcg.maps.arcgis.com'
agol_user = 'PegaLoader'
agol_password = getpass()

gis = GIS(agol_url, agol_user, agol_password)

if(debug):
    print(gis)

In [66]:
#### Check to see if the Item Exists in AGOL
query_string = 'title: Pega Cases'
item_search = gis.content.search(query_string, 'GeoJSON')

if(debug):
    print(len(item_search))

In [67]:
#### Publish the Item to AGOL\
if len(item_search) == 0:
    pega_item = gis.content.add({
        'title': 'Pega Cases',
        'tags': 'Pega, Demo, Test',
        'type': 'GeoJson',
        'typeKeywords': 'Coordinates Type, CRS, Feature, FeatureCollection, GeoJSON, Geometry, GeometryCollection'
    }, pega_temp)
    pega_fl = pega_item.publish()
else:
    if(debug):
        print("Handle item update")
    item_search[0].update(None, pega_temp)

if debug:
    display(pega_item)
    pega_map = gis.map()
    fl_search = gis.content.search(query_string, 'Feature Layer')
    pega_map.add_layer(fl_search[0])
    pega_map